In [2]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [1]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [5]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [63]:
freq = 4
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])
term = data_source[:Yeld_10Y] .- data_source[:T_Bill] 

data = merge(excess_returns, real_t_bill, dy, term, log1p.(data_source[:T_Bill]))

start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq))


,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1393,0.7471,0.0586,0.2386,-1.026,-0.5885,-0.087,0.1453,0.9413,0.421,0.0773
Price_10Y_T_Bill,0.0851,0.6398,0.0108,0.1091,-0.5762,0.5559,-0.0877,-0.0183,1.491,0.1268,-0.0128
T_Bill_CPI,0.0162,0.1019,0.0059,0.0259,-0.1073,-0.3253,-0.013,0.4055,0.9784,0.3641,0.0092
Dividend_Stock_Price,0.8084,-11.0987,-14.4962,-13.2889,-17.9466,-0.1532,-15.8611,0.9766,-0.9167,-17.9326,-14.1149
Yeld_10Y_T_Bill,0.0246,0.1588,0.0546,0.0898,-0.1224,-0.2701,0.0191,0.8068,0.0302,2.2191,0.0559
T_Bill,0.0586,0.5761,0.1649,0.2223,0.0004,0.742,0.0786,0.9548,0.8548,2.8124,0.1669


nothing

In [64]:
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill,std
Total_Stock_T_Bill,0.1355,0.1217,0.1308,1.1797,0.0277,-0.1808,-0.5434,0.067
Price_10Y_T_Bill,-0.0343,-0.1262,-0.0605,0.9065,-0.0068,0.8016,0.0603,0.0399
T_Bill_CPI,-0.0041,-0.006,0.0021,0.3007,-0.0003,0.072,0.083,0.0071
Dividend_Stock_Price,-0.1046,-0.107,-0.1413,-1.4515,0.9757,0.107,0.3704,0.0693
Yeld_10Y_T_Bill,0.0031,-0.0036,0.0151,-0.0251,0.0004,0.8235,0.014,0.0071
T_Bill,0.0015,0.0188,-0.0061,-0.0897,0.0005,0.0828,0.9791,0.0083


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
Total_Stock_T_Bill,1.0,0.14,0.05,-0.97,0.1,-0.19
Price_10Y_T_Bill,0.14,1.0,0.17,-0.12,-0.03,-0.6
T_Bill_CPI,0.05,0.17,1.0,-0.07,-0.14,0.01
Dividend_Stock_Price,-0.97,-0.12,-0.07,1.0,-0.1,0.18
Yeld_10Y_T_Bill,0.1,-0.03,-0.14,-0.1,1.0,-0.77
T_Bill,-0.19,-0.6,0.01,0.18,-0.77,1.0


In [65]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,-0.0339,0.0092,0.0196,-17.2682,-0.0227,0.1825
5,-0.005,0.0115,0.0055,-16.914,0.0224,0.1414
15,0.0191,0.0122,0.0018,-16.4446,0.0386,0.1194
25,0.028,0.0112,0.0012,-16.1833,0.0422,0.1167


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.1259,0.0669,0.0149,0.273,0.0224,0.029
5,0.0591,0.0314,0.009,0.6829,0.0268,0.0554
15,0.0315,0.0141,0.0078,0.9613,0.0177,0.0661
25,0.0238,0.0091,0.0076,1.0341,0.014,0.0673


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.0129,-0.0113,0.05,-0.0112,-0.0378,0.0327
5,0.0442,0.0336,-0.005,-0.0333,0.0108,-0.0221
15,-0.0101,0.0363,-0.0432,-0.0283,-0.0005,-0.0648
25,0.0426,-0.0214,-0.0497,-0.0265,-0.033,-0.0437


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.1491,0.0275,-0.0146,0.1239,-0.0466,-0.0047
5,0.087,0.0284,0.0199,-0.0436,-0.0387,-0.0655
15,-0.0006,-0.0174,0.0568,0.0565,0.0408,-0.1061
25,-0.0409,0.0521,0.113,-0.0278,-0.0583,-0.0199


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3319,-0.2832,-0.2414,-0.1181,-0.0339,0.0493,0.1726,0.2157,0.2657
5,-0.1416,-0.1204,-0.1024,-0.0444,-0.0055,0.0341,0.0929,0.112,0.1367
15,-0.0547,-0.0434,-0.0328,-0.0023,0.0189,0.0404,0.0708,0.0812,0.0915
25,-0.0265,-0.0179,-0.0109,0.0119,0.0279,0.0444,0.0673,0.0758,0.0843


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1491,-0.1221,-0.1005,-0.0353,0.009,0.0551,0.1172,0.1394,0.1662
5,-0.0606,-0.0491,-0.04,-0.0097,0.0111,0.0327,0.0629,0.0743,0.0851
15,-0.0207,-0.015,-0.0108,0.0027,0.0123,0.0216,0.0356,0.0399,0.0449
25,-0.0099,-0.0067,-0.0037,0.0051,0.0112,0.0174,0.0261,0.0289,0.0326


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0144,-0.0095,-0.0051,0.0097,0.0194,0.0295,0.0444,0.0488,0.0551
5,-0.016,-0.0124,-0.0092,-0.0005,0.0055,0.0115,0.0205,0.0231,0.0263
15,-0.0163,-0.0137,-0.0112,-0.0033,0.0019,0.007,0.0144,0.0168,0.0198
25,-0.0169,-0.0142,-0.0115,-0.0038,0.0013,0.0063,0.0136,0.016,0.0189


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-17.9147,-17.811,-17.7157,-17.448,-17.2689,-17.0867,-16.8211,-16.7276,-16.6155
5,-18.5254,-18.2633,-18.0384,-17.378,-16.9087,-16.4477,-15.7916,-15.5802,-15.3327
15,-18.7357,-18.3322,-18.0306,-17.0756,-16.4501,-15.8009,-14.8823,-14.5717,-14.1938
25,-18.5797,-18.2008,-17.8857,-16.8786,-16.1771,-15.4803,-14.4868,-14.1799,-13.8016


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0746,-0.0666,-0.0596,-0.0382,-0.0225,-0.0071,0.014,0.0206,0.0286
5,-0.0398,-0.0301,-0.022,0.0042,0.0225,0.0405,0.0664,0.0747,0.085
15,-0.0033,0.0041,0.0096,0.0267,0.0387,0.0505,0.0675,0.0731,0.0801
25,0.0092,0.0146,0.0193,0.0326,0.0422,0.0516,0.0654,0.0695,0.0738


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.1159,0.1265,0.1353,0.1625,0.1822,0.2021,0.2311,0.2398,0.2516
5,0.0102,0.0325,0.0509,0.1037,0.1418,0.1798,0.2317,0.248,0.2672
15,-0.0345,-0.0116,0.0096,0.0748,0.1202,0.1649,0.2279,0.2478,0.2697
25,-0.0421,-0.0157,0.0041,0.0719,0.1175,0.1615,0.2273,0.2474,0.27


In [66]:
real_equity_scenarios = reshape(scenarios[1, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)
real_bond_scenarios = reshape(scenarios[2, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)

ret_in_years = cum_returns_in_periods(vcat(real_equity_scenarios, real_bond_scenarios), periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names[1:2], string.(periods))

for a in 1:2
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  



,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-0.0143,0.0288
5,0.0005,0.017
15,0.0209,0.0141
25,0.0293,0.0124


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.1284,0.0717
5,0.0593,0.0309
15,0.0282,0.0127
25,0.0194,0.0091


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0162,-0.0036
5,0.0398,0.0369
15,-0.0236,0.0162
25,0.0436,-0.0143


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.1368,0.0225
5,0.0899,0.0519
15,-0.01,-0.0362
25,-0.0468,0.0336


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3163,-0.2714,-0.2262,-0.0999,-0.0144,0.0698,0.1983,0.2427,0.2928
5,-0.136,-0.1165,-0.0965,-0.0389,0.0001,0.0394,0.0984,0.1172,0.1409
15,-0.0457,-0.0343,-0.0251,0.0019,0.0207,0.0399,0.0674,0.0756,0.0857
25,-0.0151,-0.0082,-0.0025,0.016,0.0291,0.0424,0.0613,0.0674,0.0754


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1393,-0.1104,-0.088,-0.0197,0.0291,0.0773,0.1451,0.1697,0.1955
5,-0.0542,-0.0433,-0.0333,-0.0036,0.0168,0.0377,0.0681,0.0779,0.0902
15,-0.0153,-0.0102,-0.0067,0.0055,0.0139,0.0226,0.0349,0.0389,0.0434
25,-0.009,-0.0055,-0.0024,0.0063,0.0125,0.0185,0.0271,0.0303,0.0335


In [67]:
mdd, mddl = max_drawdown_and_length(real_equity_scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(real_bond_scenarios[1,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.263932  0.362644  0.432472  0.508697  0.646195

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 18.0  30.0  40.0  55.0  88.0

:Price_10Y_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.184713  0.25526  0.308437  0.373665  0.504509

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 21.0  34.0  45.0  59.0  83.0